In [2]:
import datetime
from functools import reduce
import regex as re
import warnings
import arrow
import json
import os
import requests
import numpy as np
import math
from google.cloud import bigquery
import pandas as pd
from datetime import datetime as dt

In [2]:
df=pd.DataFrame({'alias':['abc','def','ghi','jkl']})

In [3]:
print(df)

  alias
0   abc
1   def
2   ghi
3   jkl


In [7]:
def __alias_substrings(df, in_list, col_name='alias'):
    df=df[df[col_name].str.contains('|'.join(in_list), case=False)]
    return df

In [11]:
in_list=['bc','kl']
result= __alias_substrings(df,in_list)
print(result)

  alias
0   abc
3   jkl


In [46]:
    df=pd.DataFrame({'alias':['abc','def','ghi','jkl']})
    in_list=['a_bc','j_kl']
    groups = in_list
    if in_list is not None:
        filtered_aliases = []
        substrings = []
        for group_id in groups:
            print(group_id)
            substrings.append(group_id.split('_'))
        unique_aliases = df.columns
        print(unique_aliases)
        unique_aliases = list(filter(None, unique_aliases))
        print(unique_aliases)
    else:
        remove_cols = ['time_local']
        filtered_aliases = list(set(df.columns.to_list()) - set(remove_cols))
    print (filtered_aliases)

a_bc
j_kl
Index(['alias'], dtype='object')
['alias']
[]


In [8]:
df=pd.DataFrame({'alias':['a_bc','def','ghi','jkl']})
in_list=['a_bc','j_kl','heatPump_enable_sensor', 'chiller_enable_sensor']

#def __alias_substrings(df, in_list, col_name='alias'):
def __alias_substrings(df, in_list, col_name='alias'):
        groups = in_list
    
        if in_list is not None:
            filtered_aliases = []
            substrings = []
            # Creating list of substrings
            for group_id in groups:
                print("group id == > ",group_id)
                substrings.append(group_id.split('_'))
                print(substrings)
            unique_aliases = df.columns
            print(unique_aliases)
            # Removing None from the list of unique aliases
            unique_aliases = list(filter(None, unique_aliases))
            # Filter the aliases that are to be used for the KPI
            for alias in unique_aliases:
                for sub in substrings:
                    if all(a.strip() in alias for a in sub):
                        #print(alias)
                        filtered_aliases.append(alias)
        else:
            remove_cols = ['time_local']
            filtered_aliases = list(set(df.columns.to_list()) - set(remove_cols))
        return filtered_aliases
   

In [9]:
result= __alias_substrings(df,in_list)
print(result)

group id == >  a_bc
[['a', 'bc']]
group id == >  j_kl
[['a', 'bc'], ['j', 'kl']]
group id == >  heatPump_enable_sensor
[['a', 'bc'], ['j', 'kl'], ['heatPump', 'enable', 'sensor']]
group id == >  chiller_enable_sensor
[['a', 'bc'], ['j', 'kl'], ['heatPump', 'enable', 'sensor'], ['chiller', 'enable', 'sensor']]
Index(['alias'], dtype='object')
[]


In [123]:
set([1,1,2,1,1,2])

{1, 2}

In [14]:
import pandas as pd

# create a sample dataframe
df = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [5, None, None, 8],
    'C': [9, 10, None, 12]
})

# check for missing values in each column
missing_values = df.isna().sum()

print(missing_values)


A    0
B    2
C    1
dtype: int64


In [154]:
import pandas as pd
df= pd.DataFrame({'A':[1,2,3], 'B': [4,5,6], 'C':[7,8,9]})
print(df)

   A  B  C
0  1  4  7
1  2  5  8
2  3  6  9


In [158]:
rows_column = df.loc[[0,2],['A','C','B']]

In [159]:
rows_column

,A,C,B
0,1,7,4
2,3,9,6


In [163]:
ext_data = df.loc[[0,2]]

In [166]:
ext_data1=df.loc[:,'A']

In [167]:
ext_data1

0    1
1    2
2    3
Name: A, dtype: int64

In [23]:
df=pd.DataFrame({"value":['True','False','True','True','False','False']})

In [24]:
df.loc[df["value"] == True, "value"] = 1
df.loc[df["value"] == False, "value"] = 0 

In [25]:
print(df.loc[:,'value'])

0     True
1    False
2     True
3     True
4    False
5    False
Name: value, dtype: object


In [26]:
df['expected_streak'] = df.value.ne(df.value.shift())
print(df['expected_streak'])

0     True
1     True
2     True
3    False
4     True
5    False
Name: expected_streak, dtype: bool


In [27]:
# get timelocal for the start of streak
df['streak_time'] = df.loc[df.expected_streak == True]['time_local']
#print(df['streak_time'])

KeyError: 'time_local'

In [ ]:
# convert streak start time to datetime
df['streak_time'] = pd.to_datetime(df['streak_time'])
# converte time_local to datetime
df['time_local'] = pd.to_datetime(df['time_local'])

In [212]:
# fill null values from previous values by using forward filling
df = df.fillna(method='ffill')
print(df)

   value  expected_streak
0   True             True
1  False             True
2   True             True
3   True            False
4  False             True
5  False            False


In [214]:
# calculate delta in minutes
df['delta'] = (df['time_local'] - df['streak_time']
                ).dt.total_seconds() / 60
return df

In [221]:
df_eq=pd.DataFrame({"equation":['alias_a > 90']})

In [222]:
def apply_equation_1(self, df=None, equation=None, temp=None):
        """ This function basically takes a dataframe and an equation
        and then tries to apply this equation and results in the form of boolean.
        Parameters
        ----------
        df: Dataframe
        A Dataframe for an aliases or a pair of aliases
        equation: String
        This string has a set of instruction
        Returns
        -------
        df: Dataframe
        """
        def func(x):
            y = eval(equation)
            return y
        if df.shape[0] > 0:
            df['time_local'] = pd.to_datetime(df['time_local'])
            # in case equation is given and not missing_values or constant values
            if equation is not None:
                # iterate for given given alias columns in the config file
                for col in self.__res:
                    # add dataframe with alias column
                    if col in equation:
                        col_name = f"x['{col}']"
                        equation = equation.replace(col, col_name)
                # time and date based conditions
                if 'day' in equation:
                    df['day'] = df.time_local.dt.dayofweek
                    equation = equation.replace('day', 'x["day"]')
                if 'date' in equation:
                    df['date'] = df.time_local.dt.date
                    equation = equation.replace('date', 'str(x["date"])')
                if 'time' in equation:
                    df['time'] = df.time_local.apply(lambda x: x.strftime('%H:%M'))
                    equation = equation.replace('time', 'str(x["time"])')
                if 'hour' in equation:
                    df['hour'] = df.time_local.dt.hour
                    equation = equation.replace('hour', 'x["hour"]')
                if 'minute' in equation:
                    df['minute'] = df.time_local.dt.minute
                    equation = equation.replace('minute', 'x["minute"]')
                if 'month' in equation:
                    df['month'] = df.time_local.dt.month
                    equation = equation.replace('month', 'x["month"]')
                # apply equation
                df['value'] = df.apply(func, axis=1)
            # in case equation is None
            else:
                df['value'] = df['alias_a']
        return df

In [226]:
df = apply_equation_1(
    self, df, equation=rule['equation_1'], temp=True).reset_index(drop=True)

NameError: name 'self' is not defined

In [225]:
def func(x):
    y = eval(equation)
    return y

In [229]:
pd.__version__

'1.5.2'

In [282]:
##Pandas Series Object
data= pd.Series([0.25,0.5,0.75,1])
data

#values as simple NumPy arrays
data.values

#index is an array like object of type pd.index
data.index

data[1]
data[:3]
data[::3]


#Series as generalized NumPy array
data=pd.Series([0.25, 0.5, 0.75, 1.00], index=['a','b','c','d'])
data


data=pd.Series([0.25,0.5,0.75,1], index=['a','b','c','d'])
data

data['b']

#Series as generalized NumPy array
data=pd.Series([0.25,0.5,0.75,1], index=[2,4,6,8])
data

#Series as specialized dictionary
population_dict={ 'California':38332521,
                 'Texas' : 25656565,
                 'New York' : 12132121,
                 'Florida' : 23232323,
                 'Illinois' :15142122    
}
population = pd.Series(population_dict)
population

population['Texas']

pd.Series([2,4,6])

pd.Series(5, index=[100,200,300])
pd.Series(8, index=[100,200,300])


#data can be a dictionary, in which index defaults to the sorted dictionary keys
pd.Series({2:'a', 1:'b', 3:'c'})

pd.Series({2:'a', 1:'b', 3:'C'}, index=[2,3])

2    a
3    C
dtype: object

In [300]:
#The Pandas DataFrame Object

#DataFrame as a generalized NumPy array
area_dict={'California': 426484,
          'Texas': 145644,
           'New York':459684,
            'Florida': 850215,
             'Illinois':859564}
area=pd.Series(area_dict)
area

states=pd.DataFrame({'population':population,
                    'area': area})
states

states.index
states.columns


#DataFrame as specialized dictionary
states['area']


#Constructing DataFrame objects
#From a single Series object
pd.DataFrame(population, columns=['population'])

#From a list of dicts
data = [{'a': i, 'b': 2*i}
           for i in range(3)]
data
pd.DataFrame(data)

data= [{'a':i, 'b' :2*i}
          for i in range(3)]
pd.DataFrame(data)

#if we have missing values pd give NaN(not a number)
pd.DataFrame([{'a':1, 'b':2}, {'b':3, 'c':4}])


#From a dictionary of Series objects
pd.DataFrame({'population': population,
                'area': area})

,population,area
California,38332521,426484
Texas,25656565,145644
New York,12132121,459684
Florida,23232323,850215
Illinois,15142122,859564


In [360]:
#Handling Missing Data
vals=np.array([1, None, 3, 4])
vals
#vals.sum() will give error

#NaN: Missing numerical data
vals1=np.array([1, np.nan, 3, 4])
vals1
vals1.dtype

1+np.nan

0*np.nan

vals1.sum(), vals1.min(), vals1.max()


np.nansum(vals1), np.nanmin(vals1), np.nanmax(vals1)

pd.Series([1, np.nan, 3, None])

vals2=pd.Series(range(5), dtype='int')
vals2
vals2[0]=[None]
vals2

#in Pandas, string data is always stored with an object dtype.
#isnull(): Generate a boolean mask indicating missing values
#notnull(): Opposite of isnull()
#dropna(): Return a filtered version of the data
#fillna(): Return a copy of the data with missing values filled or imputed


#Pandas data structures have two useful methods for detecting null data: isnull() and notnull(). Either one will return a 
#Boolean mask over the data. For example:
x=pd.Series([1, np.nan, 'Hello', None])
x
x.isnull()

x[x.notnull()]

x.dropna()

df=pd.DataFrame([[1, np.nan, 2],
                [2,3,5],
                [np.nan, 4, 6]])
df

df.dropna()

df.dropna(axis = 'columns')

#But this drops some good data as well; you might rather be interested in dropping rows or columns with all NA values, 
#or a majority of NA values. This can be specified through the how or thresh parameters, which allow fine control of the 
#number of nulls to allow through.

#The default is how='any', such that any row or column (depending on the axis keyword) containing a null value will be dropped. 
#You can also specify how='all', which will only drop rows/columns that are all null values:

df[3]=np.nan
df
df.dropna(axis='columns', how='all')

#For finer-grained control, the thresh parameter lets you specify a minimum number of non-null values for the row/column 
#to be kept:
df.dropna(axis ='rows', thresh=3)


#Filling null values
data=pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data
data.fillna(0)

#We can specify a forward-fill to propagate the previous value forward:
data.fillna(method='ffill')

#Or we can specify a back-fill to propagate the next values backward:
data.fillna(method='bfill')


#For dataframe
#For DataFrames, the options are similar, but we can also specify an axis along which the fills take place:
df
df.fillna(method='ffill')
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [395]:
#Hierarchical Indexing
#make use of hierarchical indexing (also known as multi-indexing) to incorporate multiple index levels within a single index. 
#In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and 
#two-dimensional DataFrame objects.

index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

pop[('California', 2010):('Texas',2000)]


#But the convenience ends there. For example, if you need to select all values from 2010, you'll need to do some messy 
#(and potentially slow) munging to make it happen:
pop[[i for i in pop.index if i[1] == 2010]]


#The Better Way: Pandas MultiIndex
index = pd.MultiIndex.from_tuples(index)
index

#reindex
pop = pop.reindex(index)
pop
#Here the first two columns of the Series representation show the multiple index values, 
#while the third column shows the data. Notice that some entries are missing in the first column: in this 
#multi-index representation, any blank entry indicates the same value as the line above it.


#Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:
pop[:, 2010]


#MultiIndex as extra dimension
#. The unstack() method will quickly convert a multiply indexed Series into a conventionally indexed DataFrame:
pop_df = pop.unstack()
pop_df


#Naturally, the stack() method provides the opposite operation:
pop_df.stack()



pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df


f_u18 = pop_df['under18'] /pop_df['total']
f_u18.unstack()


#Methods of MultiIndex Creation
#The most straightforward way to construct a multiply indexed Series or DataFrame is to simply pass a list of two or 
#more index arrays to the constructor. For example:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df


#Explicit MultiIndex constructors
#you can construct the MultiIndex from a simple list of arrays giving the index values within each level:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])


#You can construct it from a list of tuples giving the multiple index values of each point:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

#You can even construct it from a Cartesian product of single indices:
pd.MultiIndex.from_product([['a','b'],[1,2]])



#MultiIndex level names¶
#Sometimes it is convenient to name the levels of the MultiIndex. This can be accomplished by passing the names argument 
#to any of the above MultiIndex constructors, or by setting the names attribute of the index after the fact:
pop.index.names = ['state', 'year']
pop



#MultiIndex for columns
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data


#Other types of indexing and selection work as well; for example, selection based on Boolean masks:
pop[pop>20000000]



#Multiply indexed DataFrames
health_data

#Remember that columns are primary in a DataFrame, and the syntax used for multiply indexed Series applies to the columns. 
#For example, we can recover Guido's heart rate data with a simple operation:
health_data['Guido', 'HR']

#Also, as with the single-index case, we can use the loc, iloc, and ix indexers, For example:
health_data.iloc[:2, :2]

health_data.loc[:, ('Bob', 'HR')]


#slicing, IndexSlice 
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,42.0,40.0,33.0
2014,1,29.0,49.0,47.0


In [401]:
#Combining Datasets: Concat and Append

def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# example DataFrame
make_df('ABC', range(3))

df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
display('df1', 'df2', 'pd.concat([df1, df2])')



df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='col')")

'df1'

'df2'

'pd.concat([df1, df2])'

'df3'

'df4'

"pd.concat([df3, df4], axis='col')"

In [404]:
#Combining Datasets: Merge and Join
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

df3=pd.merge(df1, df2)
df3

'df1'

'df2'

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [408]:
np.array([range(i, i + 4) for i in [2, 4, 6]])

array([[2, 3, 4, 5],
       [4, 5, 6, 7],
       [6, 7, 8, 9]])

In [434]:
#Array
import numpy as np
np.random.seed(0)  # seed for reproducibility

x1 = np.random.randint(10, size=6)  # One-dimensional array
x1
x2=np.random.randint(10, size=(3,4))
x2
x3=np.random.randint(10, size=(3,4,5))
x3

#print("x3 ndim: ", x3.ndim)
#print("x3 shape:", x3.shape)
#print("x3 size: ", x3.size)
#print('dtype:', x3.dtype)


#print("itemsize:", x3.itemsize, "bytes")
#print("nbytes:", x3.nbytes, "bytes")



x2
x2_sub=x2[:2, :2]
x2_sub


#copy of arrays
x2_sub_copy=x2[:2,:2].copy()
x2_sub_copy
x2_sub_copy[0,0]=42
x2_sub_copy
x2_sub

array([[3, 5],
       [7, 6]])

In [ ]:

# get timelocal for the start of streak
df_new['streak_time'] = df_new.loc[df_new.start_of_streak == True]['time_local']

# convert streak start time to datetime
df_new['streak_time'] = pd.to_datetime(df_new['streak_time'])

# fill null values from previous values by using forward filling
df_new = df_new.fillna(method='ffill')

# calculate delta in minutes
df_new['delta'] = (df_new['time_local'] - df_new['streak_time'] 
                  ).dt.total_seconds() / 60